In [0]:
!pip install PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

fileId = drive.CreateFile({'id': '#'}) 
print(fileId['title'])  # dataset.zip
fileId.GetContentFile('temp.zip')  # Save Drive file as a local file

!unzip temp.zip -d ./

imdb_master.zip
Archive:  temp.zip
replace ./imdb_master.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ./imdb_master.csv       


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from sklearn.model_selection import train_test_split
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [0]:
df = pd.read_csv('imdb_master.csv', encoding = "ISO-8859-1")
del df['Unnamed: 0']
del df['file']
del df['type']

In [0]:
df = df.loc[0:49999]
df = df.sample(frac=1).reset_index(drop=True)

In [0]:
def clean_reviews(text):
    lemmatizer = WordNetLemmatizer()
    my_stopwords = stopwords.words('english') 
    text = text.replace("<br >", "")
    text = text.replace("</br >", "")        
    text = re.sub('[^a-zA-Z]',' ', text)
    text = text.lower() 
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
    text = [word for word in text if not word in my_stopwords]
    text = " ".join(text)
    return text  

In [0]:
df["review"]=df.review.apply(lambda x: clean_reviews(x))

In [0]:
df.head()

,review,label
0,watch movie adolescence attempt soundtrack b...,pos
1,obvious view film recommend viewer research p...,neg
2,literate intelligent drama group internationa...,pos
3,many ppl know let keep way wonder leas...,neg
4,mean come movie nice potential like run mone...,neg


In [0]:
df["label"] = df["label"].map({'pos': 1, 'neg': 0})
df.head()

,review,label
0,watch movie adolescence attempt soundtrack b...,1
1,obvious view film recommend viewer research p...,0
2,literate intelligent drama group internationa...,1
3,many ppl know let keep way wonder leas...,0
4,mean come movie nice potential like run mone...,0


In [0]:
X_train, X_test, y_train, y_test = train_test_split(df["review"], df["label"], test_size=0.2)

In [0]:
vect = TfidfVectorizer(ngram_range=(1,2), binary=True)
train_data = vect.fit_transform(X_train)
test_data = vect.transform(X_test)

In [0]:
# SVM

from sklearn.svm import LinearSVC
svm = LinearSVC()

In [0]:
svm.fit(train_data, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [0]:
y_pred_class = svm.predict(test_data)

In [0]:
metrics.accuracy_score(y_test, y_pred_class)

0.9044

In [0]:
metrics.confusion_matrix(y_test, y_pred_class)

array([[4495,  535],
       [ 421, 4549]])

In [0]:
# Logistic Regression

from sklearn.linear_model import LogisticRegression
nb = LogisticRegression()

In [0]:
nb.fit(train_data, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
y_pred_class = nb.predict(test_data)

In [0]:
metrics.accuracy_score(y_test, y_pred_class)

0.889

In [0]:
metrics.confusion_matrix(y_test, y_pred_class)

array([[4423,  607],
       [ 503, 4467]])

In [0]:
# Naive Bayes

from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [0]:
nb.fit(train_data, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [0]:
y_pred_class = nb.predict(test_data)

In [0]:
metrics.accuracy_score(y_test, y_pred_class)

0.8851

In [0]:
metrics.confusion_matrix(y_test, y_pred_class)

array([[4540,  490],
       [ 659, 4311]])

In [0]:
# Decision Tree

from sklearn.tree import DecisionTreeClassifier
nb = DecisionTreeClassifier()

In [0]:
nb.fit(train_data, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [0]:
y_pred_class = nb.predict(test_data)

In [0]:
metrics.accuracy_score(y_test, y_pred_class)

0.6933

In [0]:
metrics.confusion_matrix(y_test, y_pred_class)

array([[3534, 1496],
       [1571, 3399]])

In [0]:
# Random Forest 

from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators = 100)

In [0]:
clf.fit(train_data, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:
y_pred_class = clf.predict(test_data)

In [0]:
metrics.accuracy_score(y_test, y_pred_class)

0.8535

In [0]:
metrics.confusion_matrix(y_test, y_pred_class)

array([[4326,  704],
       [ 761, 4209]])

In [0]:
d